<a href="https://colab.research.google.com/github/Kolojo7/Google_Colab/blob/main/asl_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# Load in our data from CSV files
train_df = pd.read_csv("/content/drive/MyDrive/colab_data/data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/colab_data/data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

num_classes = 24

# Convert Pandas DataFrame to PyTorch tensor
y_train_data = torch.tensor(y_train.values, dtype=torch.long)
y_valid_data = torch.tensor(y_valid.values, dtype=torch.long)

# Normalize our image data
x_train = x_train / 255.
x_valid = x_valid / 255.

# Reshape the image data for the convolutional network (correct shape for Conv2d)
x_train = x_train.reshape(-1, 1, 28, 28)
x_valid = x_valid.reshape(-1, 1, 28, 28)

# Define data augmentation transformations for training
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip with 50% probability
    transforms.RandomRotation(degrees=15),   # Random rotation within -15 to 15 degrees
    transforms.ToTensor(),                   # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize the image (mean=0.5, std=0.5)
])

# For validation, we only need to convert to tensor and normalize (no augmentation)
valid_transforms = transforms.Compose([
    transforms.ToTensor(),                   # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize with the same mean and std
])

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)

        image = image.type(torch.float32)  # Ensure the image is in the correct tensor type
        return image, label

train_dataset = CustomDataset(data=x_train, labels=y_train_data, transform=data_transforms)
valid_dataset = CustomDataset(data=x_valid, labels=y_valid_data, transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

# Define the model (same as before)
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        # Define the layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=25, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(25)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=25, out_channels=50, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout2d(p=0.2)
        self.bn2 = nn.BatchNorm2d(50)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(in_channels=50, out_channels=100, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(100)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(100 * 3 * 3, 512)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Forward pass through the network
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.dropout1(x)
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = CustomCNN(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Equivalent to categorical cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define a metric function to calculate accuracy
def calculate_accuracy(outputs, targets):
    predicted = outputs.argmax(-1)
    correct = (predicted == targets).sum().item()
    total = targets.size(0)
    return correct / total

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    acc_all = []
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()  # Zero the gradients
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        # Get accuracy
        acc = calculate_accuracy(outputs, labels)
        acc_all.append(acc)

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    acc_val_all = []
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for images, labels in valid_loader:
            outputs = model(images)
            loss_val = criterion(outputs, labels)
            acc_val = calculate_accuracy(outputs, labels)
            acc_val_all.append(acc_val)
        acc_avg = np.array(acc_all).mean()
        acc_val_avg = np.array(acc_val_all).mean()
        print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {loss:.4f}, Training Accuracy: {acc_avg:.4f},  Validation loss: {loss_val:.4f}, Validation Accuracy: {acc_val_avg:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'asl_model.pth')

# Shutdown the kernel (optional)
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
import torch.nn.functional as F

# Load in our data from CSV files
train_df = pd.read_csv("/content/drive/MyDrive/colab_data/data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/colab_data/data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

num_classes = 24

# Convert Pandas DataFrame to PyTorch tensor
y_train_data = torch.tensor(y_train.values, dtype=torch.long)
y_valid_data = torch.tensor(y_valid.values, dtype=torch.long)

# Categorically encode y_train and y_valid using one-hot encoding
y_train = F.one_hot(y_train_data, num_classes)
y_valid = F.one_hot(y_valid_data, num_classes)

# Normalize our image data
x_train = x_train / 255.
x_valid = x_valid / 255.

# Reshape the image data for the convolutional network
x_train = x_train.reshape(-1,1,28,28)
x_valid = x_valid.reshape(-1,1,28,28)

import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        # Define the layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=25, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(25)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=25, out_channels=50, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout2d(p=0.2)
        self.bn2 = nn.BatchNorm2d(50)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=50, out_channels=100, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(100)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(100 * 3 * 3, 512)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Forward pass through the network
        B, C, W, H = x.shape
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.dropout1(x)
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = x.reshape(B, -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


num_classes = 24  # Replace with the actual number of classes
model = CustomCNN(num_classes) # Create an instance of the model

import torchvision.transforms as transforms
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

# Define data augmentation transformations
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust image color
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # Random affine transformation
    transforms.RandomResizedCrop(28, scale=(0.8, 1.0), ratio=(0.75, 1.333)),  # Randomly crop and resize images
    transforms.RandomRotation(10),  # Randomly rotate images by up to 10 degrees
    transforms.ToTensor(),  # Convert images to tensors
])

# We dont need to do augmentation for validation data
# For validation all we need to do is convert the images into tensor
valid_transforms = transforms.Compose([
    transforms.ToTensor()
])

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)

        #image = image.type(torch.float32)
        return torch.tensor(image, dtype=torch.float32), label

train_dataset = CustomDataset(data=x_train, labels=y_train_data)
valid_dataset = CustomDataset(data=x_valid, labels=y_valid_data)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Equivalent to categorical cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# Define a metric function to calculate accuracy
def calculate_accuracy(outputs, targets):
    predicted = outputs.argmax(-1)
    correct = (predicted == targets).sum().item()
    total = targets.size(0)
    return correct / total

# Training loop
from PIL import Image
num_epochs = 40
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    acc_all = []
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()  # Zero the gradients
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        # Get accuracy
        acc = calculate_accuracy(outputs,labels)
        acc_all.append(acc)

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    acc_val_all = []
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for images, labels in valid_loader:
            outputs = model(images)
            loss_val = criterion(outputs, labels)
            acc_val = calculate_accuracy(outputs,labels)
            acc_val_all.append(acc_val)
        acc_avg = np.array(acc_all).mean()
        acc_val_avg = np.array(acc_val_all).mean()
        print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {loss:.4f}, Training Accuracy: {acc_avg:.4f},  Validation loss: {loss_val:.4f}, Validation Accuracy: {acc_val_avg:.4f}')

import torch

# Assuming 'model' is your trained PyTorch model
torch.save(model.state_dict(), '/content/drive/MyDrive/colab_data/data/asl_data/asl_model3.pth')

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Mounted at /content/drive
Epoch [1/40], Train loss: 2.6643, Training Accuracy: 0.2990,  Validation loss: 2.8189, Validation Accuracy: 0.4653
Epoch [2/40], Train loss: 2.6065, Training Accuracy: 0.6174,  Validation loss: 2.5698, Validation Accuracy: 0.6312
Epoch [3/40], Train loss: 2.5710, Training Accuracy: 0.7023,  Validation loss: 2.5233, Validation Accuracy: 0.6591
Epoch [4/40], Train loss: 2.4626, Training Accuracy: 0.7359,  Validation loss: 2.5031, Validation Accuracy: 0.7086
Epoch [5/40], Train loss: 2.4740, Training Accuracy: 0.7486,  Validation loss: 2.5071, Validation Accuracy: 0.7204
Epoch [6/40], Train loss: 2.3818, Training Accuracy: 0.7521,  Validation loss: 2.5086, Validation Accuracy: 0.7255
Epoch [7/40], Train loss: 2.4375, Training Accuracy: 0.7530,  Validation loss: 2.5077, Validation Accuracy: 0.7238
Epoch [8/40], Train loss: 2.4325, Training Accuracy: 0.7855,  Validation loss: 2.5167, Validation Accuracy: 0.7606
Epoch [9/40], Train loss: 2.4403, Training Accuracy: 0

{'status': 'ok', 'restart': True}